<h1>AI-Generated VS Natural Images Classifier</h1>
This notebook demonstrates the process of building a PyTorch-based classifier between AI-Generated and Natural Images. 
The dataset utilized in this notebook is GenImage, accessible at the following link: https://github.com/GenImage-Dataset/GenImage. The dataset contains huge amount of Natural vs AI-Generated images from various generators. This notebook implements ResNet-50 Fine Tuning to build a classifier model, which has been proven to achieve the highest performance in a similar dataset (https://arxiv.org/abs/2312.08880).<br><br>
<div align=center>
<img src='visulization.png' width=750>
</div>
<p style="text-align:center;">Example of GenImage Dataset</p>

<br>                                                                                                                       
Note: The datasets provided in GenImage share similar structures. Consequently, the process outlined in this notebook can be used to create a classifier model for each dataset.
<br>

<h2>Importing Library</h2>

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
import torch.cuda  

<h2>Data Preparation</h2>
The dataset is already clean, so it doesn't require much preprocessing aside from resizing and normalization. The data is split with an 80:20 ratio for training and validation. You can modify general variables such as the dataset path, batch size, and CUDA configuration as needed.

In [3]:
#Load the Data
data_dir = 'Dataset/GAN/BigGan/train'
# Define transforms for the training and validation sets (adjust the needs)
data_transforms ={
    "train_transforms": transforms.Compose([transforms.RandomRotation(30),
                                           transforms.RandomResizedCrop(299), 
                                           transforms.RandomHorizontalFlip(), 
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], 
                                                                [0.229, 0.224, 0.225])]),
   "valid_transforms": transforms.Compose([transforms.Resize(300),
                                           transforms.CenterCrop(299),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])]), 
    "test_transforms": transforms.Compose([transforms.Resize(300),
                                           transforms.CenterCrop(299),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
}

# Set the split ratios for training and validation
train_data_ratio = 0.8
valid_data_ratio = 0.2

# Apply defined data transformation 
train_data = datasets.ImageFolder(data_dir, transform=data_transforms["train_transforms"])
valid_data = datasets.ImageFolder(data_dir, transform=data_transforms["valid_transforms"])

In [10]:
# Obtain training indices that will be used for validation 
num_train = len(train_data)
indices = list(range(num_train))

# Calculate the number of samples for training and validation
train_count = int(train_data_ratio * num_train)
valid_count = int(valid_data_ratio * num_train)

# Split the indices into training and validation
train_idx = indices[:train_count]
valid_idx = indices[train_count:train_count + valid_count]

# Print the percentage of total data each set represents
print("Training : ", len(train_idx))
print("Validation : ", valid_count)
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Define the loader
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = True,num_workers=100)
validloader = torch.utils.data.DataLoader(valid_data, batch_size = 32, sampler = valid_sampler)

Training : 544000
Validation : 136000


<h2>Defining Model</h2>
Load the ResNet-50 architecture with pre-trained weights, adapt the output layer to the current case, and define the optimizer.

In [23]:
#Load pre-trained ResNet-50
model_transfer = models.resnet50(pretrained=True)

# Check if GPU is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()
    
#Specify that each weight in the ResNet-50 architecture will be updated during training.
for param in model_transfer.parameters():
    param.requires_grad=True
    

# Add output layer. In this case the ouput is 4 classes
n_inputs = model_transfer.fc.in_features 
last_layer = nn.Linear(n_inputs, len(classes))
model_transfer.fc = last_layer

# If GPU is available, move the model to GPU
if use_cuda:
    model_transfer = model_transfer.cuda()
    
#Define and configure the optimizer as needed
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9)

for name, module in model_transfer.named_children():
    print(f"Layer Name: {name}")
    print(module)
    print("-----------------------------------")

Layer Name: conv1
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
-----------------------------------
Layer Name: bn1
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
-----------------------------------
Layer Name: relu
ReLU(inplace=True)
-----------------------------------
Layer Name: maxpool
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
-----------------------------------
Layer Name: layer1
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=

In [18]:
import torch.cuda  
torch.cuda.is_available()

True

<h2>Training Function</h2>
In this training function, we check the loss value of of training and validation data.

In [26]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    '''returns trained model'''
    # Initialize tracker for minimum validation loss
    valid_loss_min = np.inf
  
    for epoch in range(1, n_epochs+1):
        # Initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
    
        # Model training
        model.train()
        for batch_idx, (data,target) in enumerate(trainloader):
            # Move to GPU
            if use_cuda:
                data,target = data.cuda(), target.cuda()
      
            # Then, clear (zero out) the gradient of all optimized variables
            optimizer.zero_grad()
            # Forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # Perform the Cross Entropy Loss. Calculate the batch loss.
            loss = criterion(output, target)
            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Perform optimization step (parameter update)
            optimizer.step()
            # Record the average training loss
            train_loss = train_loss + ((1/ (batch_idx + 1 ))*(loss.data-train_loss))
      
        # Model validation
        model.eval()
        for batch_idx, (data,target) in enumerate(validloader):
            # Move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # Update the average validation loss
            # Forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # Calculate the batch loss
            loss = criterion(output, target)
            # Update the average validation loss
            valid_loss = valid_loss + ((1/ (batch_idx +1)) * (loss.data - valid_loss))
      
        # Print training/validation stats
        print('Epoch: {} \tTraining Loss: {:.5f} \tValidation Loss: {:.5f}'.format(
            epoch,
            train_loss,
            valid_loss))
    
        # Save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.5f} --> {:.5f}).'.format(
                  valid_loss_min,
                  valid_loss))
            
            valid_loss_min = valid_loss
        print('Saving Model...')
        torch.save(model.state_dict(), 'FineTuning10%Only.pt')
    # Return trained model
    return model

# Define loaders transfer
loaders_transfer = {'train': trainloader,
                    'valid': validloader}

# Train the model (adjust the parameters as needed)
model_transfer = train(3, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')


Epoch: 1	Training Loss: 0.08514	Validation Loss: 0.01706
Validation loss decreased (inf --> 0.01706).
Saving Model...
Epoch: 2	Training Loss: 0.05563	Validation Loss: 0.02439
Saving Model...
Epoch: 3	Training Loss: 0.05187	Validation Loss: 0.00808
Validation loss decreased (0.01706 --> 0.00808).
Saving Model...


<h2>Testing Function</h2>
We evaluate the model performance judging by the overall accuracy. You can adjust the testing data to check your models capability. In this notebook, we test the model to classify between natural and AI-Generated images created from GAN.  

In [29]:
import numpy as np
import time

def test(loaders, model1, criterion, use_cuda):
    #Monitor test loss and accuracy
    test_loss = 0.0
    correct = 0.0
    total = 0.0

    #Monitor per-class accuracy
    class_correct = np.zeros(len(loaders['test'].dataset.classes))
    class_total = np.zeros(len(loaders['test'].dataset.classes))

    model1.eval()  # set model into evaluation/testing mode
    start_time = time.time()
    #Iterating over test data
    for batch_idx, (data, target) in enumerate(loaders['test']):
        #Move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()

        model1 = model1.to(data.device)

        #Forward pass: compute predicted outputs by passing inputs to the model
        output = model1(data)

        #Convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]

        #Calculate the loss
        loss = criterion(output, target)

        #Update average test loss
        test_loss += ((1 / (batch_idx + 1)) * (loss.data - test_loss))

        #Compare predictions to ground truth
        correct += pred.eq(target.data.view_as(pred)).sum().item()
        total += data.size(0)

        #Update per-class counts
        for i in range(len(loaders['test'].dataset.classes)):
            class_correct[i] += pred[target == i].eq(target[target == i].data.view_as(pred[target == i])).sum().item()
            class_total[i] += (target == i).sum().item()
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('Overall Test Accuracy: {:.3f}% ({}/{})'.format(100. * correct / total, correct, total))

    for i, class_name in enumerate(loaders['test'].dataset.classes):
        class_acc = 100. * class_correct[i] / class_total[i] if class_total[i] != 0 else 0
        print('Accuracy for class {}: {:.3f}% ({}/{})'.format(class_name, class_acc, int(class_correct[i]), int(class_total[i])))

    execution_time = time.time() - start_time
    print(f'Execution Time: {execution_time} seconds')

print('GAN Testing Result : ') 
test(loaders_transfer1, dynamic_ensemble_model, criterion_transfer, use_cuda)

GAN Testing Result : 
Test Loss: 0.027351

Overall Test Accuracy: 98.975% (11877.0/12000.0)
Accuracy for class ai: 98.100% (5886/6000)
Accuracy for class nature: 99.850% (5991/6000)
Execution Time: 168.1136178970337 seconds
